Clean Price of anarchy
---------------------

In [1]:
import glob
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
dfs = []
for filename in glob.glob('/home/nightwing/rsc/Literature-Article/src/data/price_of_anarchy/*.json'):
    dfs.append(pd.read_json(filename))

In [3]:
anarchy_df = pd.concat(dfs, ignore_index=True)

In [4]:
anarchy_df = anarchy_df[~anarchy_df['title'].isnull()]

In [5]:
size = len(anarchy_df['title'].unique())
size

537

Cleaning the data set.
---------------------

**1. Removing articles.**
Check whether the key `price of anarchy` is within title or absract. If not remove from data set.

Create two sets, `not in title` and `not in abstract`. Remove the intersection of the two.

In [7]:
key = 'price of anarchy'

In [8]:
anarchy_df['title'] = anarchy_df['title'].str.lower()

In [9]:
not_in_title = set()
for i, row in anarchy_df.iterrows():
    if key not in row['title']:
        not_in_title.add(i)

In [10]:
not_in_abstract = set()
for i, row in anarchy_df.iterrows():
    if row['abstract']:
        if key not in row['abstract']:
            not_in_abstract.add(i)

In [11]:
drop = list(not_in_abstract.intersection(not_in_title))

In [12]:
anarchy_df = anarchy_df[~anarchy_df.index.isin(drop)]

In [13]:
size = len(anarchy_df['title'].unique())
size

296

In [14]:
len(anarchy_df['author'].unique())

668

In [20]:
anarchy_df.date.sort_values().unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018])

In [24]:
anarchy_df = anarchy_df.replace(2018, 2017)

**2. Authors.** Cleaning authors.

The Levenshtein distance is used as a mesaure of similarity. A lot of manual work was put into this.

In [16]:
from fuzzywuzzy import fuzz

In [17]:
anarchy_df['author'] = anarchy_df['author'].str.lower()

In [18]:
len(anarchy_df['author'].unique())

668

In [22]:
pairs = []
for count, i in tqdm(enumerate(anarchy_df.author.unique())):
    for j in anarchy_df.author.unique():
        ratio = fuzz.token_set_ratio(i,j)
        if ratio >=85 and ratio != 100:
            pairs.append((i, j))

668it [00:13, 51.01it/s]


In [41]:
len(pairs)

68

In [42]:
df = pd.DataFrame(pairs, columns=['first_suggestion', 'second_suggestion'])
df.head(10)

,first_suggestion,second_suggestion
0,george christodoulou,christodoulou giorgos
1,tamer basar,tamer başar
2,tomas valla,valla tomáš
3,laszlo a. vegh,végh lászló a
4,laszlo a. vegh,végh lászló a.
5,christodoulou george,christodoulou giorgos
6,schäfer guido,guido schaefer
7,jing zhang,jin zhang
8,guido schäfer,guido schaefer
9,e. altman,eitan altman


In [43]:
len(df)

68

In [48]:
df = df.drop_duplicates('first_suggestion').sort('first_suggestion')

/home/nightwing/anaconda3/envs/literature/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [49]:
len(df)

56

In [50]:
df.to_csv('/home/nightwing/rsc/Literature-Article/src/data/price_of_anarchy/pairs.csv')

In [2]:
replacements_df = pd.read_csv('/home/nightwing/rsc/Literature-Article/src/data/price_of_anarchy/names.csv')

In [3]:
replacements_df.head()

,Unnamed: 0,first_suggestion,second_suggestion,check
0,64,a. freund,ari freund,True
1,52,chen yao,yan chen,False
2,9,e. altman,eitan altman,False
3,40,eitan altman,e. altman,True
4,41,eva tardos,tardos éva,True


In [60]:
replacements_df.check = [bool(c) for c in replacements_df.check]